#### Load Modules

In [2]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))
/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.6.3, the latest is 0.6.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


#### Cross reference N1 P_Ids with other data types

In [3]:
#1. Load Wake Data and work out which P_Id's are not there in other sleep data types. 
#These need to be dropped --> p_id_to_drop
folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/'

p_id_to_drop = []

paths = joblib.load(folder + 'N1' + '_paths.pkl')
N1_groups = pd.Series(paths['s_p_id'])
N1_groups_counts = N1_groups.value_counts()

for ind, val in zip(N1_groups_counts.index, N1_groups_counts):
    #loop through the groups of each data type
    # print('Outer index is: ' + str(ind) ) for debugging
    for data_type in ['N2', 'N3','REM']:
        paths = joblib.load(folder + data_type + '_paths.pkl')
        groups = pd.Series(paths['s_p_id'])
 
        if ind in groups.values: 
            pass 
        else:
            #print(ind)
            p_id_to_drop.append(ind)
            N1_groups_counts = N1_groups_counts.drop(index = ind)
            break

print(p_id_to_drop)

#2. Work Out which P_id's are present only once in other data types but twice in Wake 
#These need to be semi dropped --> single_pids

N1_multiple_pids = N1_groups_counts[N1_groups_counts > 1]

single_pids = []

for ind, val in zip(N1_multiple_pids.index, N1_multiple_pids):
    # For each p_id that has more than one value check if it has more than one record in other data types
    
    for data_type in ['N2', 'N3','REM']:
        paths = joblib.load(folder + data_type + '_paths.pkl')
        groups = pd.Series(paths['s_p_id'])
        group_counts = groups.value_counts()
        
        if group_counts[ind] > 1: 
            pass
            
        else:
            single_pids.append(ind)
            
single_pids = list(np.unique(single_pids))

print(single_pids)

['4411', '3305']
['2206', '2209']


In [12]:
#All data types have 3 records for 2210
for data_type in ['N1' , 'N2', 'N3','REM']:
    paths = joblib.load(folder + data_type + '_paths.pkl')
    groups = pd.Series(paths['s_p_id'])
    group_counts = groups.value_counts()
    print( group_counts['2210'] )

3
3
3
3


#### Load N1 Data , drop relevant p_ids (identified above)

In [17]:
#Load wake data _______________________________________________
paths = joblib.load(folder + 'N1' + '_paths.pkl')
X = joblib.load('N1_frequency_data.pkl')

#Load groups and y labels _____________________________________
groups = pd.Series(paths['s_p_id'])
s_class_list = pd.Series(paths['s_class_list'])
y = s_class_list.map({'HC': 0 , 'PD' : 1 , 'PD+RBD' : 2 , 'RBD' : 3}) 

#Drop p_ids that are not there in all other data types
X['groups'] = groups 
X['y'] = y
X = X[~X['groups'].isin(p_id_to_drop)]

#Drop p_ids (two) that should only be present once
for val in single_pids:
    duplicates = X[X['groups'].duplicated(keep=False) & (X['groups'] == val)]
    X = X.drop(duplicates.index[0])
# X is what I want now, all features plus additional columns called 'groups' and 'y' 
X_N1 = X.copy()

X_N1.columns = [col + '_N1' for col in X_N1.columns] #For joining with other data types later this is important

#These P_ids have two  or three (2210) values
two_ind_vals = list(X_N1['groups_N1'].value_counts()[ (X_N1['groups_N1'].value_counts() > 1 ) ].index)
two_ind_vals

#X_N1 contains all regional features AND groups + y columns. 
# X_N1 has num rows 

#Sort the dataframe by p_id and reset the index
X_N1 = X_N1.sort_values(by = 'groups_N1' , ascending = True)
X_N1 = X_N1.reset_index(drop = True)

In [20]:
X_N1['groups_N1'].values

array(['1101', '1101', '1102', '1103', '1103', '1104', '1105', '1105',
       '1106', '1106', '1108', '1108', '1109', '1110', '1111', '1111',
       '1112', '1113', '1113', '1116', '1116', '1117', '1118', '1119',
       '1119', '2202', '2203', '2203', '2204', '2204', '2205', '2205',
       '2206', '2207', '2208', '2209', '2210', '2210', '2210', '2211',
       '2211', '2212', '2212', '2213', '2214', '2215', '2216', '3301',
       '3302', '3303', '3304', '3304', '3306', '4401', '4401', '4402',
       '4403', '4403', '4404', '4405', '4406', '4407', '4408', '4408',
       '4409', '4409', '4410'], dtype=object)

In [21]:
X_N1

,Delta_Central_N1,Theta_Central_N1,Low_Alpha_Central_N1,Alpha_Central_N1,High_Alpha_Central_N1,Sigma_Central_N1,Beta_Central_N1,Gamma_Central_N1,TotalAbsPow_Central_N1,Fast_ratio_Central_N1,...,Alpha_Right Temporal_N1,High_Alpha_Right Temporal_N1,Sigma_Right Temporal_N1,Beta_Right Temporal_N1,Gamma_Right Temporal_N1,TotalAbsPow_Right Temporal_N1,Fast_ratio_Right Temporal_N1,Alpha_shift_Right Temporal_N1,groups_N1,y_N1
0,0.361374,0.278515,0.217975,0.142088,0.034991,0.082963,0.106511,0.028548,30.670926,0.390253,...,0.164842,0.029249,0.070579,0.084403,0.015991,34.465101,0.375787,9.602886,1101,0
1,0.365410,0.268491,0.207587,0.133948,0.029056,0.082806,0.113701,0.035645,28.226842,0.391383,...,0.152654,0.027003,0.070624,0.090768,0.022726,23.873921,0.368447,9.298230,1101,0
2,0.412212,0.323588,0.292569,0.156997,0.023152,0.049676,0.049705,0.007823,30.410397,0.281902,...,0.155814,0.021329,0.035768,0.042968,0.013442,13.857836,0.265113,13.761902,1102,0
3,0.486820,0.242581,0.191805,0.148365,0.028025,0.050508,0.059980,0.011746,28.830325,0.287666,...,0.176632,0.033110,0.039753,0.038475,0.006526,41.208645,0.292367,7.635879,1103,0
4,0.488009,0.285536,0.184128,0.135618,0.026849,0.040587,0.043640,0.006610,45.328834,0.232453,...,0.151107,0.026719,0.039588,0.031320,0.004928,51.215546,0.236581,9.289353,1103,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,0.459211,0.162204,0.115625,0.147490,0.037211,0.097110,0.115391,0.018594,17.287396,0.426792,...,0.184109,0.048837,0.093697,0.104769,0.021620,25.696597,0.488814,2.311339,4408,1
63,0.490238,0.158357,0.110791,0.142646,0.036779,0.091376,0.102242,0.015141,18.974058,0.378297,...,0.162282,0.042002,0.077837,0.076512,0.015276,31.516548,0.359284,2.580190,4408,1
64,0.439556,0.227428,0.206057,0.136481,0.023368,0.043302,0.096496,0.056737,34.155219,0.352137,...,0.149538,0.021865,0.038293,0.063478,0.019857,16.904423,0.292657,12.913326,4409,1
65,0.344684,0.240242,0.248805,0.155348,0.021114,0.046460,0.147070,0.066196,22.359408,0.531445,...,0.183399,0.026170,0.049536,0.075783,0.028478,23.766315,0.391713,10.727807,4409,1


#### Join Non-N1 Data to N1 Data

In [22]:
folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/'

#Concatenate all dataframes together to 'Concat_df'
concat_df = X_N1.copy()

for data_type in ['N2','N3','REM']:

    #1. Load the data - put features, groups and y labels into data frame X_df
    X_df = joblib.load( data_type + '_frequency_data.pkl' )
    paths = joblib.load(folder + data_type + '_paths.pkl')
    
    groups = pd.Series(paths['s_p_id'])
    s_class_list = pd.Series(paths['s_class_list'])
    y = s_class_list.map({'HC': 0 , 'PD' : 1 , 'PD+RBD' : 2 , 'RBD' : 3}) 
    
    X_df['groups'] = groups 
    X_df['y'] = y

    #2. Keep only the groups that are also present in Wake data
    X_df_groups_reduced = X_df['groups'][X_df['groups'].isin(list(X_N1['groups_N1'].values)) ]

    #3. Drop those p_ids that are present more than they are in Wake data. E.g. once in wake data but twice here then drop one
    matching_indices = [] #keep track of the indices to keep
    series = X_df_groups_reduced
    for val in np.unique(X_N1['groups_N1']):
        if val in two_ind_vals:
            index_of_value = series[series == val].index[0]
            matching_indices.append(index_of_value)
            index_of_value = series[series == val].index[1]
            matching_indices.append(index_of_value)
            
            if val == '2210':
                index_of_value = series[series == val].index[2]
                matching_indices.append(index_of_value)      
        else:
            index_of_value = series[series == val].index[0]
            matching_indices.append(index_of_value)
        
    if len(matching_indices) == 67:
        pass
    else: 
        print('something went wrong')
        
    X_df = X_df.loc[matching_indices]
    X_df = X_df.sort_values(by = 'groups' , ascending = True)
    X_df = X_df.reset_index(drop = True)
    
    # Append data_type to column name
    X_df.columns = [col + '_' + data_type for col in X_df.columns]
    
    concat_df = pd.concat([concat_df, X_df], axis = 1)

#### Check that Pid's match across data types

In [24]:
data_types = ['N1','N2','N3','REM']
cols_to_select = ['groups_' + data_type for data_type in data_types]
print(concat_df[cols_to_select].nunique(axis = 1).eq(1).all())

cols_to_select = ['y_' + data_type for data_type in data_types]
print(concat_df[cols_to_select].nunique(axis = 1).eq(1).all())

True
True


#### Save data as appropriate

In [26]:
y = concat_df['y_REM']
groups = concat_df['groups_REM']
data_types = ['N1','N2','N3','REM']
cols_to_drop = ['groups_' + data_type for data_type in data_types] + ['y_' + data_type for data_type in data_types]
concat_df = concat_df.drop(columns = cols_to_drop)

In [27]:
joblib.dump(y , 'y_sleep_concat.pkl')
joblib.dump(groups, 'groups_sleep_concat.pkl')
joblib.dump(concat_df, 'sleep_concat_df.pkl')

['sleep_concat_df.pkl']